In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.1.3'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.co

In [2]:
# Connection to Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-09-06 22:35:09--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2022-09-06 22:35:09 (10.6 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Create spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('electronics_reviews').config('spark.driver.extraClassPath','/content/postgresql-42.2.9.jar').getOrCreate()

In [4]:
# Read in data
from pyspark import SparkFiles
src = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz'
spark.sparkContext.addFile(src)

df = spark.read.csv(SparkFiles.get('amazon_reviews_us_Electronics_v1_00.tsv.gz'), sep='\t', timestampFormat='yyyy/MM/dd', inferSchema=True, header=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [5]:
# Counting number of rows in dataset
rows = df.count()
cols = len(df.columns)
print(f'rows: {rows}, columns: {cols}')

rows: 3093869, columns: 15


In [6]:
# Verifying datatypes
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
# Creating dataframe for reviews, and all related information
review_id = df[['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date']]
review_id.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: string (nullable = true)



In [8]:
# Creating dataframe for products, with only unique values
products = df[['product_id', 'product_title']]
products = products.dropDuplicates(['product_id'])
products.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [9]:
# Creating dataframe for customer ids and count of customers
customercount = df.groupBy('customer_id').count()
customers = customercount.withColumnRenamed('count', 'customer_count')
customers.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [10]:
# Creating dataframe for vine reviews and all related information
vines = df[['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine']]
vines.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [11]:
# Changing column data types to match schema template
from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, DateType

review_id = review_id.withColumn('review_date', col('review_date').cast(DateType()))
customers = customers.withColumn('customer_count', col('customer_count').cast(IntegerType()))
vines = vines.withColumn('star_rating', col('star_rating').cast(IntegerType()))

In [ ]:
# Configure settings for RDS
mode = 'append'
jdbc_url= 'jdbc:postgresql://<endpoint>:5432/<database>'
config = {'user':'<username>', 
          'password': '<password>', 
          'driver':'org.postgresql.Driver'}

In [ ]:
# Writing review dataframe into table
review_id.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Writing product dataframe into table
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Writing customer dataframe into table
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Writing vine dataframe into table
vines.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [ ]:
# https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz
# Electronics